This program creates library format files .mrc for uploading to electronic library. 
Input data of standarts are in Excel file.
Created for Krylov State Research Centre by Mikhachev Evgeny

In [152]:
import xlrd, xlwt
import datetime
from pymarc import MARCReader
from pymarc import Record, Field

In [67]:
rb = xlrd.open_workbook('base.xls',formatting_info=True)
sheet = rb.sheet_by_index(0)

In [155]:
val = sheet.row_values(0)[0]
vals = [sheet.row_values(rownum) for rownum in range(sheet.nrows)]
print(len(vals))

5259


In [7]:
def serial_date_to_string(srl_no):
    new_date = datetime.datetime(1900,1,1,0,0) + datetime.timedelta(srl_no-2)
    return new_date.strftime("%Y-%m-%d")
date_of_input = datetime.datetime.fromtimestamp(int(vals[1][4]))

serial_date_to_string(vals[1][4])
#01.07.03


'2003-07-01'

In [203]:
with open('smeci2.mrc', 'rb') as fh:
    reader = MARCReader(fh)
    for record in reader:
        print(record)
        for f in record.get_fields('200'):
            print(f)
    

=LDR  00621nam a22001693i 450 
=001  ru\ksrc\gost\241302
=005  20191105110718.0
=029  10$aRU$b5632-2014$cГОСТ
=100  \\$a20191106h        k  y0rusy01020304ca
=101  0\$arus
=102  \\$aRU
=105  \\$ay   l   000zy
=200  1\$aГОСТ 5632-2014. Нержавеющие стали и сплавы коррозионно-стойкие, жаростойкие и жаропрочные марки$eвведен 01.01.1979
=300  \\$aИзм. №1 (ИУС 1-2019)
=801  \1$aRU$bKSRC$c20191105
=998  \\$aГО56Не0000000000
=999  \\$aКГНЦ$bКГНЦ$e$p$u0$v1

=200  1\$aГОСТ 5632-2014. Нержавеющие стали и сплавы коррозионно-стойкие, жаростойкие и жаропрочные марки$eвведен 01.01.1979
None


In [202]:
for i in range(len(vals)):
    with open('smeci2.mrc', 'rb') as fh:  # smeci2.mrc is a template for changing some fields
        reader = MARCReader(fh)
        record = next(reader)
        # field001 id is generated after upload, so we can remove it.
        for f in record.get_fields('001'):
            print(f , ' ')
            record.remove_field(f)
        #record['001'] = 'ru\\ksrc\\gost\\241303'
        #record['005'] = '20191106110718.0'
        gost_num = vals[i][3][5:]
        record['029']['b'] = gost_num
        record['200']['a'] = vals[i][3]+'. '+vals[i][8]
        if vals[i][5] =='2002-09-01':
            record['200']['e'] = ''
        else:
            record['200']['e'] = 'введен ' + serial_date_to_string(vals[2][4]).replace('-', '.')
        if not vals[i][5]:
            record['200'].add_subfield('e', '')
        else:
            record['200'].add_subfield('e', 'взамен ' +vals[i][5])
        #record['200']['e'] = 'взамен ' +vals[i][7]
        if not vals[i][7]:
            record['300']['a'] =''
        else:
            record['300']['a'] = vals[i][7]
        

        
    with open(f'gost{i}.mrc', 'wb') as out:
        out.write(record.as_marc())
    '''
    with open(f'gost{i}.mrc', 'r+', encoding="utf-8") as file_in:
        text = file_in.read()
        text = text.replace(f'"ru\ksrc\gost\"{241302}', f'"ru\ksrc\gost\"{241307+i}')   

    with open(f'gost{i}.mrc', "w", encoding="utf-8") as file_out:
        file_out.write(text)
    '''